In [1]:
import sys
import numpy
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras import layers
from numpy import array
from pickle import dump
from random import randint
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ReduceLROnPlateau


Using TensorFlow backend.


In [2]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r',encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

In [3]:
in_filename = 'gk4z_df_10-20_TXT.txt'
data = load_doc(in_filename)

In [4]:
# tokenizer = Tokenizer()
tokenizer = load(open('tokenizer_gen_6_TOKENS10-20PADDING.pkl',  'rb'))

In [5]:

corpus = data.split("\n")    
tokenizer.fit_on_texts(corpus)
vocab_size = len(tokenizer.word_index) + 1

In [6]:
print('vocab_size: ', vocab_size)
print('number of lines: ', len(corpus))
print('all the words: ', len(data.split()))

vocab_size:  19794
number of lines:  7206
all the words:  101689


In [7]:
input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range (1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)


In [8]:
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

19

In [9]:
input_sequences = array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [10]:
input_sequences.shape

(94483, 19)

In [11]:

X,y = input_sequences[:,:-1],input_sequences[:,-1]


In [12]:
y = np_utils.to_categorical(y, num_classes=vocab_size)

In [13]:
y.shape

(94483, 19794)

In [14]:
seq_length = X.shape[1]
seq_length

18

In [15]:
model = load_model('model__gen_6_TOKENS10-20PADDING.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.


The learned embedding needs to know the size of the vocabulary and the length of input sequences as previously discussed. It also has a parameter to specify how many dimensions will be used to represent each word. That is, the size of the embedding vector space.

Common values are 50, 100, and 300. We will use 50 here, but consider testing smaller or larger values.

We will use a two LSTM hidden layers with 100 memory cells each. More memory cells and a deeper network may achieve better results.

A dense fully connected layer with 100 neurons connects to the LSTM hidden layers to interpret the features extracted from the sequence. The output layer predicts the next word as a single vector the size of the vocabulary with a probability for each word in the vocabulary. A softmax activation function is used to ensure the outputs have the characteristics of normalized probabilities.

In [16]:
model = Sequential()
model.add(layers.Embedding(vocab_size, 50, input_length=seq_length, mask_zero = True))
model.add(LSTM(300, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(300))
model.add(Dropout(0.1))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 18, 50)            989700    
_________________________________________________________________
lstm_1 (LSTM)                (None, 18, 300)           421200    
_________________________________________________________________
dropout_1 (Dropout)          (None, 18, 300)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 300)               721200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 300)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 19794)             5957994   
Total params: 8,090,094
Trainable params: 8,090,094
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [18]:
filepath="weights-TOKENS-10-20PADDINGimprovement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [19]:

reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=1,  min_lr=0.000001, verbose=1,cooldown=1)
callbacks_list = [checkpoint, reduce_lr]

In [19]:
model.fit(X, y, epochs=2, callbacks=callbacks_list)

Epoch 1/2
 1408/94483 [..............................] - ETA: 1:07:32 - loss: 9.3908

KeyboardInterrupt: 

In [40]:
# save the model to file
model.save('model__gen_6_TOKENS10-20PADDING.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer_gen_6_TOKENS10-20PADDING.pkl', 'wb'))

In [74]:
model.load_weights('weights-TOKENS-10-20PADDINGimprovement-29-1.3047.hdf5')
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [29]:
# weights-TOKENS-10-20PADDINGimprovement-11-3.8014.hdf5

# weights-TOKENS-10-20PADDINGimprovement-16-2.6663.hdf5

# weights-TOKENS-10-20PADDINGimprovement-21-1.9680.hdf5
        
# weights-TOKENS-10-20PADDINGimprovement-29-1.3047.hdf5      
        

token_list  [11509]
pad_token_list  [[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0 11509]]
predicted  [1]
в
token_list  [11509]
pad_token_list  [[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0 11509]]
predicted  [1]
в
token_list  [11509]
pad_token_list  [[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0 11509]]
predicted  [1]
в
token_list  [11509]
pad_token_list  [[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0 11509]]
predicted  [1]
в
token_list  [11509]
pad_token_list  [[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0 11509]]
predicted  [1]
в
token_list  [11509]
pad_token_list  [[    0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0     0     0 11509]]
predicted  [1]


In [70]:
def generate_text(seed_text, n_words, max_sequence_len, model):
#     print(seed_text)
    result = ''
    for j in range(n_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
#         print('token_list ', token_list)
        token_list = pad_sequences([token_list], maxlen= 
                             max_sequence_len-1, padding='pre')
#         print('pad_token_list ', token_list)
        predicted = model.predict_classes(token_list, verbose=0)
#         print('predicted ', predicted)
  
        output_word = ""
        
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                result += " " + output_word
#                 print('output_word ', output_word)
                break
        seed_text += " " + output_word
        
#         print('seed_text ', seed_text)
    return result

In [66]:
generate_text('ариана ты выиграла грэмми приходи за статуэткой ох нет она в своих фирменных наушниках она нас не слышит', 10, max_sequence_len, model)

' их трек и еще прямо во сне уже с афтепати'

In [71]:
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
	result = list()
	in_text = seed_text
	# generate a fixed number of words
	for _ in range(n_words):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
	return ' '.join(result)

In [75]:
for i in range (10):
  seed_text = corpus[randint(0,len(corpus))]
  print(seed_text)
  generated = generate_seq(model, tokenizer, seq_length, seed_text, 7)
  print(generated+ '\n')

может уже эминема наконец ну и бринги давно уже были пора бы ещё раз
пора вроде пора бы песня из сетлиста

если я послушаю слитый альбом до релиза буду ли я грешником
бы грешником был ставь быть можно ли

сколько фоток и видосов с одного концерта можно выложить в инстаграм чтобы не начали отписываться
начали на нем и случайно любят смерти

30 секунд до джаред ты ебаный пенсионер кто тебе в уши насрал сука пиздец
без собой название фестивалей не был ее

долго пытался найти тёрнера где этот залаченнный франт что это за поздний леннон блин
дайте поздний голос похож на концерты с

ну он вроде бы не приезжал в россию а пост направлен на непонятный вброс без конкретного примера
себе работает станет нет билеты на любой

ну их и так выделяют как отдельные жанры чо за вопрос
вот когда ты понравиться мутится получше в

кому на бесплатном концерте заплатить за билет чтобы избежать всего этого безумия
безумия был ставь лайк когда тоже устанут

это годнота на самом деле скажу как человек такое не с

In [73]:
for i in range (10):
  seed_text = corpus[randint(0,len(corpus))]
  print(seed_text)
  generated = generate_text(seed_text, 10,  max_sequence_len, model)
  print(generated+ '\n')

2016 еще не наступил но я уже коплю деньги на концерты я зависима или нет
 делаю нормально ли это программа сестра в фб подвох приделывать

мама сказала найти символ 2017 года на застолье подойдёт ли фотка ури
 парикмахеру для меня не убирает его в чгкз уроки разножанровые

жизненноменя сразу бомбит хотя здравый смысл подсказывает что у каждого свой вкус но чёт это не помогает
 станет взросление или говорит что не выпендриваться твою названий группы

за неделю 3 поста опубликовали моих я просто в шоке нахожусь если честно
 слушали и манит не разрезая их баллы детей быстрее лет

оливер сайкс заболел после концерта в киеве заметим ли мы разницу в вокале
 заметим ли это идиоты репетицией получается петь как бывшей шутку

не нашел себя ни на одной фотке с концерта какой яд более действенный
 его написано и согласились здоровья не так сделала и бывшей

знают ли девочки пытающиеся вписаться на концерты об ориентации музыкантов заранее или бывает нежданчик
 недостойной или успеха на сенсейшна

In [ ]:
generate_text(seed_text, n_words, max_sequence_len, model):